In [7]:
# Create sample sentences
prediction_file = 'data/predicted_sentences.txt'
reference_file = 'data/references.txt'
with open(prediction_file, 'w') as f:
    f.write('aaa\tThe University of California San Diego has been ranked the 17th best university in the world.\nbbb\tUp four spots compared to last year.\n')
with open(reference_file, 'w') as f:
    f.write('aaa\tThe University of California San Diego has been ranked the 17th best university in the world.\naaa\tThe University of California San Diego has been ranked the 17th best university in the world.\nbbb\tUp four spots compared to last year.\nbbb\tUp four spots compared to last year.\n')

In [8]:
import matplotlib 
matplotlib.use('Agg')  # to solve the backend problem when there's no GUI
import getopt 
import hashlib
import io 
import json 
import os 
import sys 
sys.path.append('./coco-caption/') 
from pycocotools.coco import COCO 
from pycocoevalcap.eval import COCOEvalCap 

In [9]:
class CocoAnnotations:
  def __init__(self):
    self.images = [];
    self.annotations = [];
    self.img_dict = {};
    licenses = [{
                "id" : "text",
                "name" : "test",
                "url" : "test",
                }]
    self.res = {"info" : {},
                "type" : 'captions',  # should keep type as captions [another choice is istances, though I haven't check]
                "images" :  self.images,
                "annotations" : self.annotations,
                "licenses" : licenses,
                }

  
  def get_image_dict(self, img_name):
    image_hash = int(int(hashlib.sha256(img_name).hexdigest(), 16) % sys.maxint)
    if image_hash in self.img_dict:
      assert self.img_dict[image_hash] == img_name, 'hash colision: {0}: {1}'.format(image_hash, img_name)
    else:
      self.img_dict[image_hash] = img_name
    image_dict = {"id" : image_hash,
                "width" : 0,
                "height" : 0,
                "file_name" : img_name,
                "license" : '',
                "url" : img_name,
                "date_captured" : '',
                }
    return image_dict,image_hash

  def read_file(self,filename):
    count = 0
    with open(filename,'r') as opfd:
      for line in opfd:
        count +=1
        id_sent = line.strip().split('\t')
        assert len(id_sent)==2
        sent = id_sent[1].decode('ascii', 'ignore')
        image_dict,image_hash = self.get_image_dict(id_sent[0])
        self.images.append(image_dict)

        self.annotations.append({
          "id" : len(self.annotations)+1,
          "image_id" : image_hash,
          "caption" : sent,
          })

        if count%1000 == 0:
          print 'Processed %d ...' % count

  def dump_json(self, outfile):
    self.res["images"] = self.images  # should keep it
    self.res["annotations"] = self.annotations
    res = self.res
    with io.open(outfile, 'w', encoding='utf-8') as fd:
      fd.write(unicode(json.dumps(res, ensure_ascii=True,sort_keys=True,indent=2,separators=(',', ': '))))
    
class CocoResFormat:
  def __init__(self):
    self.res = []
    self.caption_dict = {}

  def read_file(self, filename):
    count = 0
    with open(filename,'r') as opfd:
      for line in opfd:
        count +=1
        id_sent = line.strip().split('\t')
        sent = id_sent[1].decode('ascii', 'ignore')
        assert len(id_sent) == 2
        img_id = int(int(hashlib.sha256(id_sent[0]).hexdigest(), 16) % sys.maxint)
        imgid_sent = {}

        if img_id in self.caption_dict:
          assert self.caption_dict[img_id] == sent
        else:
          self.caption_dict[img_id] = sent
          imgid_sent['image_id'] = img_id
          imgid_sent['caption'] = sent
          self.res.append(imgid_sent)
        if count%1000 == 0:
          print 'Processed %d ...' % count

  def dump_json(self, outfile):
    res = self.res
    with io.open(outfile, 'w', encoding='utf-8') as fd:
      fd.write(unicode(json.dumps(res,
         ensure_ascii=False,sort_keys=True,indent=2,separators=(',', ': '))))

In [10]:
# Assigning output json path
reference_json = '{0}.json'.format(reference_file)
prediction_json = '{0}.json'.format(prediction_file)

# load reference and predicts 
crf = CocoAnnotations()
crf.read_file(reference_file)
crf.dump_json(reference_json)

crf = CocoResFormat()
crf.read_file(prediction_file)
crf.dump_json(prediction_json)

# create coco object and cocoRes object.
coco = COCO(reference_json)
cocoRes = coco.loadRes(prediction_json)

# create cocoEval object.
cocoEval = COCOEvalCap(coco, cocoRes)

# evaluate results
cocoEval.evaluate()

# print output evaluation scores
for metric, score in cocoEval.eval.items():
    print '%s: %.3f'%(metric, score)

loading annotations into memory...
0:00:00.001273
creating index...
index created!
Loading and preparing results...     
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'reflen': 23, 'guess': [23, 21, 19, 17], 'testlen': 23, 'correct': [23, 21, 19, 17]}
ratio: 0.999999999957
Bleu_1: 1.000
Bleu_2: 1.000
Bleu_3: 1.000
Bleu_4: 1.000
computing METEOR score...
METEOR: 1.000
computing Rouge score...
ROUGE_L: 1.000
computing CIDEr score...
CIDEr: 10.000
CIDEr: 10.000
Bleu_4: 1.000
Bleu_3: 1.000
Bleu_2: 1.000
Bleu_1: 1.000
ROUGE_L: 1.000
METEOR: 1.000
